# Label genereting

In [ ]:
import numpy as np
import pandas as pd

ev = pd.read_csv('gms_eval_v3.csv')
ev = ev[['ID', 'action', '1차 평가']]

In [2]:
ev = ev[ev['ID'].str[0] == "B"]

In [3]:
action1 = ev[ev['action'].str[4] == "1"]
action2 = ev[ev['action'].str[4] == "2"]
action3 = ev[ev['action'].str[4] == "3"]
action4 = ev[ev['action'].str[4] == "4"]

In [4]:
action1 = action1.rename(columns={"action":"action1", "1차 평가":"action1_평가"})
action2 = action2.rename(columns={"action":"action2", "1차 평가":"action2_평가"})
action3 = action3.rename(columns={"action":"action3", "1차 평가":"action3_평가"})
action4 = action4.rename(columns={"action":"action4", "1차 평가":"action4_평가"})

In [5]:
# tmp dataframe to make a pid list
df_concat = pd.concat([action1, action2, action3, action4], sort=False)

In [6]:
pidLi = np.array(df_concat['ID'].sort_values().unique())

In [7]:
prev_arr = pd.DataFrame(columns=['ID', 'action1', 'action1_평가', 'action2', 'action2_평가','action3', 'action3_평가','action4', 'action4_평가'])

for item in pidLi:
    
    arr1 = action1[action1['ID'] == item].reset_index().drop(columns=['index'])
    arr2 = action2[action2['ID'] == item].drop(columns=['ID']).reset_index().drop(columns=['index'])
    arr3 = action3[action3['ID'] == item].drop(columns=['ID']).reset_index().drop(columns=['index'])
    arr4 = action4[action4['ID'] == item].drop(columns=['ID']).reset_index().drop(columns=['index'])

    cat_arr = pd.concat([arr1, arr2, arr3, arr4], axis=1)
    new_arr = pd.concat([prev_arr, cat_arr], axis=0)
    prev_arr = new_arr

In [8]:
#final concatenated output
prev_arr

,ID,action1,action1_평가,action2,action2_평가,action3,action3_평가,action4,action4_평가
0,B001,GMS_1_1,3.0,GMS_2_1,3.0,GMS_3_1,3.0,GMS_4_1,3.0
1,B001,GMS_1_2,3.0,GMS_2_2,3.0,GMS_3_2,3.0,GMS_4_2,3.0
2,B001,GMS_1_3,3.0,GMS_2_3,3.0,GMS_3_3,3.0,GMS_4_3,3.0
3,B001,GMS_1_4,3.0,GMS_2_4,2.0,GMS_3_4,3.0,GMS_4_4,3.0
4,B001,GMS_1_5,3.0,GMS_2_5,3.0,GMS_3_5,2.0,GMS_4_5,3.0
...,...,...,...,...,...,...,...,...,...
0,B205,GMS_1_1,3.0,GMS_2_1,3.0,GMS_3_1,2.0,GMS_4_1,3.0
1,B205,GMS_1_2,3.0,GMS_2_2,3.0,GMS_3_2,2.0,GMS_4_2,3.0
2,B205,GMS_1_3,3.0,GMS_2_3,3.0,GMS_3_3,2.0,GMS_4_3,3.0
3,B205,GMS_1_4,3.0,GMS_2_4,3.0,GMS_3_4,2.0,GMS_4_4,3.0


In [9]:
dnx = prev_arr
# dnx = prev_arr.dropna(how='any')

In [10]:
gdnx = dnx.groupby('ID').mean()
gdnx = gdnx.assign(label_pid=0)

In [11]:
for idx in gdnx.index:
    cnt = 0
    if (gdnx.loc[idx]['action1_평가'] < 2.0):
        cnt += 1
    if (gdnx.loc[idx]['action2_평가'] < 2.0):
        cnt += 1
    if (gdnx.loc[idx]['action3_평가'] < 2.0):
        cnt += 1
    if (gdnx.loc[idx]['action4_평가'] < 2.0):
        cnt += 1
    # print(cnt)
    if (cnt > 1):
        gdnx.loc[idx, 'label_pid'] = 1.0

In [12]:
gdnx['label_pid'].value_counts()

0    88
1    53
Name: label_pid, dtype: int64

In [13]:
gdnx

,action1_평가,action2_평가,action3_평가,action4_평가,label_pid
ID,,,,,
B001,3.000000,2.833333,2.833333,3.000000,0
B006,2.400000,3.000000,1.600000,1.800000,1
B007,3.000000,3.000000,2.000000,2.600000,0
B011,3.000000,3.000000,2.400000,1.400000,0
B014,1.666667,1.000000,0.333333,0.000000,1
...,...,...,...,...,...
B201,2.000000,2.000000,2.000000,0.000000,0
B202,3.000000,3.000000,3.000000,2.666667,0
B203,2.666667,3.000000,2.666667,2.666667,0


In [19]:
final = gdnx[['label_pid']]

In [24]:
final.to_csv('development_status_label.csv', index=True)